In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
dataset_path = '../data/DS_ML Coding Challenge Dataset.xlsx'
train_dataset = pd.read_excel(dataset_path, sheet_name='Training Dataset')
test_dataset = pd.read_excel(dataset_path, sheet_name='Test Dataset')

In [3]:
a = train_dataset.copy()
b = test_dataset.copy()

In [4]:
train_dataset = a.copy()
test_dataset = b.copy()

In [5]:
def preprocess_data(dataset):
    '''
    Returns X and y after converting categorical variables to one-hot encoding and creating time features
    '''
    
    # Renaming column and removing spaces
    dataset.rename(columns={'ProductType':'ProductName'}, inplace=True)
    dataset.columns = [column_name.replace(' ','') for column_name in dataset.columns]
    
    # Creating time features
    dataset['Year'] = pd.DatetimeIndex(dataset['MonthofSourcing']).year
    dataset['Month'] = pd.DatetimeIndex(dataset['MonthofSourcing']).month
    
    # Creating one-hot-encoding for categorical variables
    dataset = pd.get_dummies(dataset, columns=['ProductName'], drop_first=True, prefix='ProductName')
    dataset = pd.get_dummies(dataset, columns=['Manufacturer'], drop_first=True, prefix='Manufacturer')
    dataset = pd.get_dummies(dataset, columns=['AreaCode'], drop_first=True, prefix='AreaCode')
    dataset = pd.get_dummies(dataset, columns=['SourcingChannel'], drop_first=True, prefix='SourcingChannel')
    dataset = pd.get_dummies(dataset, columns=['ProductSize'], drop_first=True, prefix='ProductSize')
    dataset = pd.get_dummies(dataset, columns=['ProductType'], drop_first=True, prefix='ProductType')
    
    # Creating X and y
    X = dataset.drop(['MonthofSourcing','SourcingCost'], axis=1).values
    y = dataset['SourcingCost'].values
    
    return X, y

In [12]:
X_train, y_train = preprocess_data(train_dataset)
X_test, y_test = preprocess_data(test_dataset)

KeyError: "None of [Index(['ProductType'], dtype='object')] are in the [columns]"

In [13]:
from keras.preprocessing.sequence import TimeseriesGenerator
n_input = 32
n_features = 56
generator = TimeseriesGenerator(X_train, y_train, length=n_input, batch_size=1)

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [15]:
# define model
model = Sequential()
model.add(LSTM(150, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 150)               124200    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 151       
Total params: 124,351
Trainable params: 124,351
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(generator)

 31978/550144 [>.............................] - ETA: 1:43:10 - loss: 209374.1261